In [1]:
import numpy as np
import pandas as pd

In [2]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline
from plot_functions import *
from sklearn.model_selection import KFold

In [3]:
from create_matrix import *
import os
from tensor_custom_core import *
from create_matrix import *
from tensor_custom_core import *
from degree_days import dds
import sys


appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}
APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
region = "SanDiego"
year = 2014




def un_normalize(x, maximum, minimum):
    return (maximum - minimum) * x + minimum

In [4]:
def get_tensor(df, dfc):
    start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()

    static_cols = ['area', 'total_occupants', 'num_rooms']
    static_df = df[static_cols]
    static_df = static_df.div(static_df.max())
    weather_values = np.array(dds[2014][region][start - 1:stop - 1]).reshape(-1, 1)

    dfc = df.copy()

    df = dfc[energy_cols]
    col_max = df.max().max()
    col_min = df.min().min()
    # df = (1.0 * (df - col_min)) / (col_max - col_min)
    tensor = df.values.reshape((len(df), 7, stop - start))
    M, N, O = tensor.shape
    return tensor

In [5]:
au_df, au_dfc = create_matrix_single_region("Austin", year)
au_tensor = get_tensor(au_df, au_dfc)
static_au = au_df[['area','total_occupants','num_rooms']].copy()
static_au['area'] = static_au['area'].div(4000)
static_au['total_occupants'] = static_au['total_occupants'].div(8)
static_au['num_rooms'] = static_au['num_rooms'].div(8)
static_au = static_au.values

In [6]:
sd_df, sd_dfc = create_matrix_single_region("SanDiego", year)
sd_tensor = get_tensor(sd_df, sd_dfc)
static_sd = sd_df[['area','total_occupants','num_rooms']].copy()
static_sd['area'] = static_sd['area'].div(4000)
static_sd['total_occupants'] = static_sd['total_occupants'].div(8)
static_sd['num_rooms'] = static_sd['num_rooms'].div(8)
static_sd = static_sd.values

In [7]:
import pickle

def load_obj(name):
    with open(os.path.expanduser('~/git/' + name + '.pkl'), 'rb') as f:
        return pickle.load(f)

In [12]:
pred = {}
for method in ['normal', 'transfer']:
    pred[method] = {}
    for train_iter in range(500, 2100, 500):
        pred[method][train_iter] = {}
        for algo in ['adagrad', 'gd']:
            pred[method][train_iter][algo] = {}
            for static_fac in ['static', 'None']:
                pred[method][train_iter][algo][static_fac] = {}
                for lam in [0.01, 0.1, 0, 1]:
                    lam = float(lam)
                    pred[method][train_iter][algo][static_fac][lam] = {}
                    for random_seed in range(10):
                        pred[method][train_iter][algo][static_fac][lam][random_seed] = load_obj("pred_explore/pred_" + method
                                                                                               + "_" + str(train_iter)
                                                                                               + "_" + algo
                                                                                               + "_" + static_fac
                                                                                               + "_" + str(lam)
                                                                                                +"_" + str(random_seed))

In [10]:
out_transfer = load_obj("out_explore/out_transfer_500_adagrad_static_0.01")
out_normal = load_obj("out_explore/out_normal_500_adagrad_static_0.01")

In [ ]:
from common import compute_rmse_fraction
out = {}
for method in ['normal', 'transfer']:
    out[method] = {}
    for train_iter in range(500, 2100, 500):
        out[method][train_iter] = {}
        for algo in ['adagrad', 'gd']:
            out[method][train_iter][algo] = {}
            for static_fac in ['static', 'None']:
                out[method][train_iter][algo][static_fac] = {}
                for lam in [0.01, 0.1, 0, 1]:
                    lam = float(lam)
                    out[method][train_iter][algo][static_fac][lam] = {}
                    for random_seed in range(10):
                        out[method][train_iter][algo][static_fac][lam][random_seed] = {}
                        

In [ ]:
from common import compute_rmse_fraction
out = {}
for method in ['normal', 'transfer']:
    out[method] = {}
    for train_iter in range(500, 2100, 500):
        out[method][train_iter] = {}
        for algo in ['adagrad', 'gd']:
            out[method][train_iter][algo] = {}
            for static_fac in ['static', 'None']:
                out[method][train_iter][algo][static_fac] = {}
                for lam in [0.01, 0.1, 0, 1]:
                    lam = float(lam)
                    out[method][train_iter][algo][static_fac][lam] = {}
                    for random_seed in range(10):
                        out[method][train_iter][algo][static_fac][lam][random_seed] = {}
                        for appliance in APPLIANCES_ORDER[1:]:
                            print (method, train_iter, algo, static_fac, lam, random_seed, appliance)
                            out[method][train_iter][algo][static_fac][lam][random_seed][appliance] = {}
                            for f in range(10,110,10):
                                s = pd.concat(pred[method][train_iter][algo][static_fac][lam][random_seed][appliance][f]).ix[sd_df.index]
                                if appliance=="hvac":
                                    out[method][train_iter][algo][static_fac][lam][random_seed][appliance][f] = compute_rmse_fraction(appliance,s[range(4, 10)],'SanDiego')[2]
                                else:   
                                    out[method][train_iter][algo][static_fac][lam][random_seed][appliance][f] = compute_rmse_fraction(appliance, s,'SanDiego')[2]

normal 500 adagrad static 0.01 0 hvac


/Users/xox/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:20: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


normal 500 adagrad static 0.01 0 fridge
normal 500 adagrad static 0.01 0 mw
normal 500 adagrad static 0.01 0 dw
normal 500 adagrad static 0.01 0 wm
normal 500 adagrad static 0.01 0 oven
normal 500 adagrad static 0.01 1 hvac
normal 500 adagrad static 0.01 1 fridge
normal 500 adagrad static 0.01 1 mw
normal 500 adagrad static 0.01 1 dw
normal 500 adagrad static 0.01 1 wm
normal 500 adagrad static 0.01 1 oven
normal 500 adagrad static 0.01 2 hvac
normal 500 adagrad static 0.01 2 fridge
normal 500 adagrad static 0.01 2 mw
normal 500 adagrad static 0.01 2 dw
normal 500 adagrad static 0.01 2 wm
normal 500 adagrad static 0.01 2 oven
normal 500 adagrad static 0.01 3 hvac
normal 500 adagrad static 0.01 3 fridge
normal 500 adagrad static 0.01 3 mw
normal 500 adagrad static 0.01 3 dw
normal 500 adagrad static 0.01 3 wm
normal 500 adagrad static 0.01 3 oven
normal 500 adagrad static 0.01 4 hvac
normal 500 adagrad static 0.01 4 fridge
normal 500 adagrad static 0.01 4 mw
normal 500 adagrad static 0.

/Library/Python/2.7/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
common.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  aggregate_df.loc[r_i, c_i] = np.NaN


0.01 9 wm
normal 500 adagrad static 0.01 9 oven
normal 500 adagrad static 0.1 0 hvac
normal 500 adagrad static 0.1 0 fridge
normal 500 adagrad static 0.1 0 mw


In [6]:
pred_normal = load_obj("pred_normal_200_adagrad_static_0.0_0")
pred_transfer = load_obj("pred_transfer_200_adagrad_static_0.0_0")